$\newcommand{\xbf}{{\bf x}}
\newcommand{\ybf}{{\bf y}}
\newcommand{\wbf}{{\bf w}}
\newcommand{\Ibf}{\mathbf{I}}
\newcommand{\Xbf}{\mathbf{X}}
\newcommand{\Rbb}{\mathbb{R}}
\newcommand{\vec}[1]{\left[\begin{array}{c}#1\end{array}\right]}
$

# Introduction à la librairie PyTorch -- Partie 2
Matériel de cours rédigé par Pascal Germain, 2019
************

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import aidecours

## Le module `torch.nn`

In [ ]:
import torch

Le module `nn` de la librairie `torch` contient plusieurs outils pour construire l'architecture d'un réseau de neurone.

In [ ]:
from torch import nn

Reprenons l'exemple des moindres carrés de la partie précédente, afin de montrer comment exprimer le problème sous la forme d'une réseau de neurones avec les outils qu'offrent *pyTorch*.

#### Préparation des données
Préparons les données d'apprentissage sous la forme de *tenseurs pyTorch*.

In [ ]:
x = np.array([(1,1),(0,-1),(2,.5)])
y = np.array([-1., 3, 2])

x_tensor = torch.tensor(x, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [ ]:
x_tensor

In [ ]:
y_tensor

In [ ]:
y_tensor = y_tensor.unsqueeze(1) # Les méthodes du module torch.nn sont conçues pour manipuler des matrices
y_tensor

#### Couche linéaire

La classe `Linear` correspond à une *couche* linéaire. La méthode des moindres carrés nécessite seulement un neurone de sortie. 

In [ ]:
nn.Linear?

In [ ]:
neurone = nn.Linear(2, 1, bias=False)
neurone

In [ ]:
neurone.weight

In [ ]:
neurone(x_tensor)

#### Fonction de perte

In [ ]:
nn.MSELoss?

In [ ]:
perte_quadratique = nn.MSELoss()

In [ ]:
perte_quadratique(neurone(x_tensor), y_tensor)

## Module d'optimisation `torch.optim`

In [ ]:
torch.optim.SGD?

In [ ]:
eta = 0.4
alpha = 0.1

neurone = nn.Linear(2, 1, bias=False)
optimiseur = torch.optim.SGD(neurone.parameters(), lr=eta, momentum=alpha)

for t in range(20):

    y_pred = neurone(x_tensor)                   # Calcul de la sortie de la neurone
    loss = perte_quadratique(y_pred, y_tensor)   # Calcul de la fonction de perte
    loss.backward()                              # Calcul des gradients
    optimiseur.step()                            # Effectue un pas de la descente de gradient
    optimiseur.zero_grad()                       # Remet à zero les variables du gradient
    
    print(t, loss.item())

Joignons tout cela ensemble afin de réécrire le module `moindres_carres` avec les outils de *pyTorch*.

In [ ]:
class moindres_carres:
    def __init__(self, eta=0.4, alpha=0.1, nb_iter=50, seed=None):
        # Initialisation des paramètres de la descente en gradient
        self.eta = eta         # Pas de gradient
        self.alpha = alpha     # Momentum
        self.nb_iter = nb_iter # Nombre d'itérations
        self.seed = seed       # Germe du générateur de nombres aléatoires
        
        # Initialisation des listes enregistrant la trace de l'algorithme
        self.w_list = list()   
        self.obj_list = list()
        
    def _trace(self, w, obj):
        self.w_list.append(np.array(w.squeeze().detach()))
        self.obj_list.append(obj.item())      
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

        n, d = x.shape
        self.neurone = nn.Linear(d, 1, bias=False)
        perte_quadratique = nn.MSELoss()
        optimiseur = torch.optim.SGD(self.neurone.parameters(), lr=self.eta, momentum=self.alpha)
                   
        for t in range(self.nb_iter + 1):
            y_pred = self.neurone(x)
            perte = perte_quadratique(y_pred, y)         
            self._trace(self.neurone.weight, perte)
  
            if t < self.nb_iter:
                perte.backward()
                optimiseur.step()
                optimiseur.zero_grad()
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        with torch.no_grad():
            pred = self.neurone(x)
            
        return pred.squeeze().numpy()

In [ ]:
eta = 0.4      # taille du pas
alpha = 0.0    # momentum
nb_iter = 20   # nombre d'itérations

algo = moindres_carres(eta, alpha, nb_iter, seed=None)
algo.apprentissage(x, y)

In [ ]:
algo.prediction(x)

In [ ]:
w_opt = np.linalg.inv(x.T @ x) @ x.T @ y
fig, axes = plt.subplots(1, 2, figsize=(14.5, 4))
fonction_objectif = lambda w: np.mean((x @ w - y) ** 2)
aidecours.show_2d_trajectory(algo.w_list, fonction_objectif, ax=axes[0])
aidecours.show_learning_curve(algo.obj_list, ax=axes[1], obj_opt=fonction_objectif(w_opt))

## Ajout d'une couche cachée

In [ ]:
couche_cachee = nn.Linear(2, 4)
couche_cachee

In [ ]:
couche_cachee.weight

In [ ]:
couche_cachee.bias

In [ ]:
for variables in couche_cachee.parameters():
    print(variables)
    print('---')

In [ ]:
couche_cachee(x_tensor)

#### Fonctions d'activations
Fonction d'activation *ReLU*

In [ ]:
nn.ReLU?

In [ ]:
activation_relu = nn.ReLU()

In [ ]:
a = torch.linspace(-2, 2, 5)
a

In [ ]:
activation_relu(a)

In [ ]:
activation_relu(couche_cachee(x_tensor))

Fonction d'activation *tanh*

In [ ]:
nn.Tanh?

In [ ]:
activation_tanh = nn.Tanh()

In [ ]:
activation_tanh(a)

In [ ]:
activation_tanh(couche_cachee(x_tensor))

Fonction d'activation *sigmoïdale*

In [ ]:
nn.Sigmoid?

In [ ]:
activation_sigmoide = nn.Sigmoid()

In [ ]:
activation_sigmoide(a)

In [ ]:
activation_sigmoide(couche_cachee(x_tensor))

#### Succession de couches et de fonctions d'activations

In [ ]:
nn.Sequential?

In [ ]:
model = nn.Sequential(
    torch.nn.Linear(2, 4),
    torch.nn.ReLU(),
)

In [ ]:
model(x_tensor)

In [ ]:
model = nn.Sequential(
    torch.nn.Linear(2, 4),
    torch.nn.ReLU(),
    torch.nn.Linear(4, 1),
)

In [ ]:
model(x_tensor)

In [ ]:
for variables in model.parameters():
    print(variables)
    print('---')

## Réseau de neurones à une couche cachée

In [ ]:
class reseau_regression:
    def __init__(self, nb_neurones=4, eta=0.4, alpha=0.1, nb_iter=50, seed=None):
        # Architecture du réseau
        self.nb_neurones = nb_neurones # Nombre de neurones sur la couche cachée
        
        # Initialisation des paramètres de la descente en gradient
        self.eta = eta         # Pas de gradient
        self.alpha = alpha     # Momentum
        self.nb_iter = nb_iter # Nombre d'itérations
        self.seed = seed       # Germe du générateur de nombres aléatoires
        
        # Initialisation des listes enregistrant la trace de l'algorithme
        self.w_list = list()   
        self.obj_list = list()
        
    def _trace(self, obj):
        self.obj_list.append(obj.item())      
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

        n, d = x.shape
        self.model = nn.Sequential(
            torch.nn.Linear(d, self.nb_neurones),
            torch.nn.ReLU(),
            torch.nn.Linear(self.nb_neurones, 1)
        )
        
        perte_quadratique = nn.MSELoss()
        optimiseur = torch.optim.SGD(self.model.parameters(), lr=self.eta, momentum=self.alpha)
                   
        for t in range(self.nb_iter + 1):
            y_pred = self.model(x)
            perte = perte_quadratique(y_pred, y)         
            self._trace(perte)
  
            if t < self.nb_iter:
                perte.backward()
                optimiseur.step()
                optimiseur.zero_grad()
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        with torch.no_grad():
            pred = self.model(x)
            
        return pred.squeeze().numpy()

In [ ]:
nb_neurones = 4
eta = 0.1      # taille du pas
alpha = 0.1    # momentum
nb_iter = 50   # nombre d'itérations

x = np.array([(1,1),(0,-1),(2,.5)])
y = np.array([-1., 3, 2])

algo = reseau_regression(nb_neurones, eta, alpha, nb_iter, seed=None)
algo.apprentissage(x, y)

aidecours.show_learning_curve(algo.obj_list)
predictions = algo.prediction(x)
print('y    =', y)
print('R(x) =', predictions)

## Exercice

L'objectif de cet exercice est d'adapter la classe `reseau_regression` présentée plus haut pour résoudre le problème de *classification* suivant.





In [ ]:
from sklearn.datasets import make_circles
xx, yy = make_circles(n_samples=100, noise=.1, factor=0.2, random_state=10)
aidecours.show_2d_dataset(xx, yy)

Nous vous demandons de compléter la fonction `fit` de la classe `reseau_classification` ci-bas. Nous vous conseillons de vous inspirer de la régression logistique en utilisant une fonction d'activation *sigmoïdale* en sortie, ainsi que la perte du **négatif log vraisemblance**. Il n'est pas nécessaire d'ajouter un terme de régularisation au réseau.

**Notez bien**: La fonction de perte du **négatif log vraisemblance** vue en classe correspond à la classe `nn.BCELoss`

In [ ]:
class reseau_classification:
    def __init__(self, nb_neurones=4, eta=0.4, alpha=0.1, nb_iter=50, seed=None):
        # Architecture du réseau
        self.nb_neurones = nb_neurones # Nombre de neurones sur la couche cachée
        
        # Initialisation des paramètres de la descente en gradient
        self.eta = eta         # Pas de gradient
        self.alpha = alpha     # Momentum
        self.nb_iter = nb_iter # Nombre d'itérations
        self.seed = seed       # Germe du générateur de nombres aléatoires
        
        # Initialisation des listes enregistrant la trace de l'algorithme
        self.w_list = list()   
        self.obj_list = list()
        
    def _trace(self, obj):
        self.obj_list.append(obj.item())      
        
    def apprentissage(self, x, y):
        if self.seed is not None:
            torch.manual_seed(self.seed)
        
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

        n, d = x.shape
        self.model = nn.Sequential(
            torch.nn.Linear(d, self.nb_neurones),
            # Compléter l'architecture ici
        )
        
        perte_logistique = nn.BCELoss()
        optimiseur = torch.optim.SGD(self.model.parameters(), lr=self.eta, momentum=self.alpha)
                   
        for t in range(self.nb_iter + 1):
            pass # Compléter l'apprentissage ici
                
    def prediction(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        
        with torch.no_grad():
            pred = self.model(x)
            
        pred = pred.squeeze()
        return np.array(pred > .5, dtype=np.int)

Exécuter le code suivant pour tester votre réseau. Varier les paramètres pour mesurer leur influence.

In [ ]:
nb_neurones = 10
eta = 0.6     # taille du pas
alpha = 0.4   # momentum
nb_iter = 50  # nombre d'itérations

algo = reseau_classification(nb_neurones, eta, alpha, nb_iter)
algo.apprentissage(xx, yy)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
aidecours.show_learning_curve(algo.obj_list, ax=axes[0])
aidecours.show_2d_predictions(xx, yy, algo.prediction, ax=axes[1]);

Finalement, nous vous suggérons d'explorer le comportement du réseau en:
1. Modifiant la fonction d'activation *ReLU* pour une fonction d'activation *tanh*
2. Ajoutant une ou plusieurs autres couches cachées